In [1]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface.chat_models import ChatHuggingFace
from dotenv import load_dotenv
from pydantic_settings import BaseSettings
load_dotenv("./.env", override=True)

True

In [2]:
class HuggingFaceConfig(BaseSettings):
    hf_token: str
    repo_id: str
    provider: str
    temperature: float 

config = HuggingFaceConfig()

In [3]:
class HuggingFaceChatModel(ChatHuggingFace):
    def __init__(self, config: HuggingFaceConfig):
        llm = HuggingFaceEndpoint(
            repo_id=config.repo_id,
            provider=config.provider,
            huggingfacehub_api_token=config.hf_token,
            temperature=config.temperature,
        )
        super().__init__(llm=llm)

In [4]:
from langgraph.graph import StateGraph, START, END
from typing import Optional
from pydantic import BaseModel

### State that flow across nodes

In [5]:
class LoanState(BaseModel):
    application: dict= {}
    validation_error: Optional[list] = None
    credit_profile: Optional[dict] = None
    score: Optional[dict] = None
    rule_flags: list = []
    decision: Optional[str] = None
    rationale: Optional[str] = None
    require_human_approval: bool = False

### Helper Tool Calls

In [6]:
import random
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [7]:
def validate_application(state: LoanState) -> LoanState:
    # Dummy validation logic
    errors = []
    if 'income' not in state.application or state.application['income'] <= 0:
        errors.append("Income must be a positive number.")
    if 'loan_amount' not in state.application or state.application['loan_amount'] <= 0:
        errors.append("Loan amount must be a positive number.")
    state.validation_error = errors if errors else None
    return state

def fetch_credit_profile(state: LoanState) -> LoanState:
    # Dummy credit profile fetching logic
    state.credit_profile = {
        "credit_score": random.randint(300, 850),
        "existing_debts": random.randint(0, 50000)
    }
    return state

def score_application(state: LoanState) -> LoanState:
    # Dummy scoring logic
    if state.credit_profile:
        state.score = {
            "risk_score": 750 - (state.application.get('loan_amount', 0) / 1000) - (state.credit_profile['existing_debts'] / 100)
        }
    return state

def apply_business_rules(state: LoanState) -> LoanState:
    # Dummy business rules application logic
    if state.score and state.score['risk_score'] < 600:
        state.rule_flags.append("Low risk score")
    if state.application.get('loan_amount', 0) > (state.application.get('income', 0) * 5):
        state.rule_flags.append("Loan amount exceeds 5x income")
    return state

def decide(state: LoanState) -> LoanState:
    # Dummy decision logic
    if state.validation_error:
        state.decision = "Rejected"
        state.rationale = "Validation errors present."
    elif state.rule_flags:
        state.decision = "Requires Human Approval"
        state.require_human_approval = True
        state.rationale = "Business rule flags present."
    else:
        state.decision = "Approved"
        state.rationale = "All checks passed."
    return state

## LLM MOdel

In [8]:
class LLM:
    def __init__(self):
        self.llm = HuggingFaceChatModel(config)

    def generate_rationale(self, state: LoanState) -> LoanState:
        system_prompt = SystemMessagePromptTemplate.from_template(
            "You are a loan approval assistant. Based on the provided application details, credit profile, score, and rule flags, generate a concise rationale for the loan decision."
        )
        human_prompt = HumanMessagePromptTemplate.from_template(
            """Application Details: {application}
            Credit Profile: {credit_profile}
            Score: {score}
            Rule Flags: {rule_flags}
            Decision: {decision}
            Rationale:"""
        )
        prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
        formatted_prompt = prompt.format_messages(
            application=state.application,
            credit_profile=state.credit_profile,
            score=state.score,
            rule_flags=state.rule_flags,
            decision=state.decision
        )
        response = self.llm(formatted_prompt)
        state.rationale = response.content
        return state

## Build the graph(DAG)

In [9]:
graph = StateGraph(LoanState)

# Add nodes (correct syntax)
graph.add_node("validation_node", validate_application)
graph.add_node("credit_node", fetch_credit_profile)
graph.add_node("score_node", score_application)
graph.add_node("business_node", apply_business_rules)
graph.add_node("decision_node", decide)
graph.add_node("rationale_node", LLM().generate_rationale)

# Set entry point
graph.add_edge(START, "validation_node")

# Add conditional edges for proper flow control
graph.add_edge(
    "validation_node",
    "credit_node"
)

graph.add_edge(
    "credit_node",
    "score_node"
)

graph.add_edge(
    "score_node",
    "business_node"
)

graph.add_edge(
    "business_node",
    "decision_node"
)

graph.add_edge(
    "decision_node",
    "rationale_node"
)

# Final edge to end
graph.add_edge("rationale_node", END)

# Compile the graph
app = graph.compile()
print("✅ Loan approval workflow graph compiled successfully!")

✅ Loan approval workflow graph compiled successfully!


In [14]:
# Testing
initial_application = {
    "applicant_name": "John Doe",
    "income": 60000,
    "loan_amount": 200000,
    "loan_purpose": "Home Purchase"
}

# Create initial state with the application data
initial_state = LoanState(application=initial_application)

print("🏦 Starting Loan Approval Process...")
print(f"📝 Application: {initial_application}")
print("-" * 50)

try:
    # Run the workflow with the initial state
    final_state = app.invoke(initial_state)

    print(final_state)

    print("✅ Workflow completed!")
    print(f"🎯 Decision: {final_state['decision']}")
    print(f"💭 Rationale: {final_state['rationale']}")
    print(f"📊 Credit Profile: {final_state['credit_profile']}")
    print(f"⚠️  Rule Flags: {final_state['rule_flags']}")
    print(f"👤 Human Approval Required: {final_state['require_human_approval']}")

except Exception as e:
    print(f"❌ Error during workflow execution: {e}")
    import traceback
    traceback.print_exc()

🏦 Starting Loan Approval Process...
📝 Application: {'applicant_name': 'John Doe', 'income': 60000, 'loan_amount': 200000, 'loan_purpose': 'Home Purchase'}
--------------------------------------------------
{'application': {'applicant_name': 'John Doe', 'income': 60000, 'loan_amount': 200000, 'loan_purpose': 'Home Purchase'}, 'validation_error': None, 'credit_profile': {'credit_score': 656, 'existing_debts': 28066}, 'score': {'risk_score': 269.34}, 'rule_flags': ['Low risk score'], 'decision': 'Requires Human Approval', 'rationale': 'Rationale: The loan application from John Doe requires human approval due to a low risk score of 269.34, which is flagged as a concern. Despite having a credit score of 656 and an income of $60,000, the low risk score indicates potential risks that need further evaluation. The loan purpose is for a home purchase, and the applicant has existing debts of $28,066, which may impact their ability to repay the requested loan amount of $200,000. A human review is 